In [1]:
# !pip -q install earthengine-api pandas numpy scikit-learn
# Authentication from terminal is required before tunning this notebook


In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-anapaolagomes')

In [3]:

# MODIS land cover (IGBP classes in band "LC_Type1")
MODIS_LC = ee.ImageCollection("MODIS/061/MCD12Q1")


def modis_lc_for_year(year: int, region: ee.Geometry) -> ee.Image:
    return (
        MODIS_LC.filter(ee.Filter.calendarRange(year, year, "year"))
        .first()
        .clip(region)
    )


def is_forest_igbp(lc_type1: ee.Image) -> ee.Image:
    """
    Forest mask for IGBP land cover classes (strict forest):
      1 Evergreen Needleleaf Forests
      2 Evergreen Broadleaf Forests
      3 Deciduous Needleleaf Forests
      4 Deciduous Broadleaf Forests
      5 Mixed Forests

    Input: lc_type1 is the MODIS "LC_Type1" band image (integer classes).
    Output: boolean ee.Image mask (True where forest).
    """
    return (
        lc_type1.eq(1)
        .Or(lc_type1.eq(2))
        .Or(lc_type1.eq(3))
        .Or(lc_type1.eq(4))
        .Or(lc_type1.eq(5))
    )


In [4]:
ROADS_BR = ee.FeatureCollection("projects/sat-io/open-datasets/GRIP4/Central-South-America")

def frontier_features_for_year(t_year: int, region: ee.Geometry, scale: int) -> ee.Image:
    lc_t = modis_lc_for_year(t_year, region).select("LC_Type1")
    forest = is_forest_igbp(lc_t).rename("forest").toByte()
    nonforest = forest.Not().rename("nonforest").toByte()

    pix_m = ee.Number(lc_t.projection().nominalScale())

    # Distance to nearest non-forest (within forest pixels)
    dist_nf = (
        nonforest.selfMask()
        .fastDistanceTransform(256)
        .sqrt()
        .multiply(pix_m)
        .rename("dist_to_nonforest_m")
        .updateMask(forest)
    )

    # Distance to roads (meters); limit search radius for speed (e.g. 100km)
    dist_rd = (
        ROADS_BR.filterBounds(region)
        .distance(searchRadius=100000)
        .rename("dist_to_road_m")
    )

    return dist_nf.addBands(dist_rd)


In [34]:

# ===================== CONFIG =====================
ROI = ee.Geometry.Rectangle([-63.5, -10.5, -61.5, -8.5]) 
SCALE = 500
SEED = 42

# Train on these years t (label is loss in (t, t+1])
TRAIN_YEARS = [2018, 2019, 2020]

# Stratified sample sizes per year (balanced)
N_POS_PER_YEAR = 5000
N_NEG_PER_YEAR = 5000

# Unbiased eval (forest-only) year t
UNBIASED_EVAL_YEAR = 2022
N_UNBIASED_FOREST_PIXELS = 30000 

# 2 options
#   False = forest(t)=1 and forest(t+1)=0
#   True  = "stable" label: pos requires forest(t-1)&forest(t)&nonforest(t+1); neg requires forest(t-1)&forest(t)&forest(t+1)
USE_STABLE_LABEL = False


AEF_IC = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")
MODIS_LC = ee.ImageCollection("MODIS/061/MCD12Q1")  # yearly landcover

def aef_for_year(year: int, region: ee.Geometry) -> ee.Image:
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, "year")
    return AEF_IC.filterDate(start, end).filterBounds(region).mosaic().clip(region)


def label_basic_loss(t_year: int, region: ee.Geometry) -> ee.Image:
    lc_t  = modis_lc_for_year(t_year, region).select("LC_Type1")
    lc_t1 = modis_lc_for_year(t_year + 1, region).select("LC_Type1")
    forest_t  = is_forest_igbp(lc_t)
    forest_t1 = is_forest_igbp(lc_t1)

    y = forest_t.And(forest_t1.Not()).rename("label").toByte()
    valid = lc_t.mask().And(lc_t1.mask())
    return y.updateMask(valid)

def label_stable_loss(t_year: int, region: ee.Geometry) -> ee.Image:
    # pos = forest(t-1)=1 & forest(t)=1 & forest(t+1)=0
    # neg = forest(t-1)=1 & forest(t)=1 & forest(t+1)=1
    lc_tm1 = modis_lc_for_year(t_year - 1, region).select("LC_Type1")
    lc_t   = modis_lc_for_year(t_year, region).select("LC_Type1")
    lc_tp1 = modis_lc_for_year(t_year + 1, region).select("LC_Type1")

    f_tm1 = is_forest_igbp(lc_tm1)
    f_t   = is_forest_igbp(lc_t)
    f_tp1 = is_forest_igbp(lc_tp1)

    pos = f_tm1.And(f_t).And(f_tp1.Not())
    neg = f_tm1.And(f_t).And(f_tp1)

    y = pos.rename("label").toByte()  # 1 where pos
    y = y.where(neg, 0)               # 0 where neg

    keep = pos.Or(neg)
    valid = lc_tm1.mask().And(lc_t.mask()).And(lc_tp1.mask())
    return y.updateMask(keep).updateMask(valid)

def label_for_year(t_year: int, region: ee.Geometry) -> ee.Image:
    return label_stable_loss(t_year, region) if USE_STABLE_LABEL else label_basic_loss(t_year, region)

def sampling_image_for_year(t_year: int, region: ee.Geometry) -> ee.Image:
    X = aef_for_year(t_year, region)                 # A00..A63
    F = frontier_features_for_year(t_year, region, SCALE)  # dist bands
    y = label_for_year(t_year, region)               # label
    return X.addBands(F).addBands(y)

def stratified_samples_for_year(t_year: int, region: ee.Geometry,
                                n_neg: int, n_pos: int,
                                scale: int, seed: int) -> ee.FeatureCollection:
    img = sampling_image_for_year(t_year, region)

    fc = img.stratifiedSample(
        numPoints=1,                       # required even when using classPoints
        classBand="label",
        classValues=[0, 1],
        classPoints=[n_neg, n_pos],
        region=region,
        scale=scale,
        seed=seed + t_year,
        dropNulls=True,
        tileScale=4,
        geometries=True                    # keep geometry => exports .geo
    )
    return fc.map(lambda f: f.set({"tYear": t_year}))

def forest_mask_for_year(t_year: int, region: ee.Geometry) -> ee.Image:
    lc_t = modis_lc_for_year(t_year, region).select("LC_Type1")
    forest = is_forest_igbp(lc_t).rename("forest").toByte()
    return forest.updateMask(lc_t.mask())

def unbiased_forest_samples(t_year: int, region: ee.Geometry,
                            n_pixels: int, scale: int, seed: int) -> ee.FeatureCollection:
    img = sampling_image_for_year(t_year, region)  # AEF + label
    forest = forest_mask_for_year(t_year, region)

    img_forest = img.updateMask(forest)

    fc = img_forest.sample(
        region=region,
        scale=scale,
        numPixels=n_pixels,
        seed=seed + 999,
        tileScale=4,
        geometries=True
    ).map(lambda f: f.set({"tYear": t_year, "unbiased": 1}))

    return fc

def export_fc_to_drive(fc: ee.FeatureCollection, description: str, filename_prefix: str, selectors: list[str]):
    task = ee.batch.Export.table.toDrive(
        collection=fc.select(selectors),
        description=description,
        fileNamePrefix=filename_prefix,
        fileFormat="CSV"
    )
    task.start()
    return task

# ===================== BUILD + EXPORT =====================

# Determine embedding band names once (ensures correct ordering)
bands = aef_for_year(TRAIN_YEARS[0], ROI).bandNames()
band_list = bands.getInfo()
print("AEF bands:", band_list[:8], "... total:", len(band_list))
frontier_band_list = ["dist_to_nonforest_m", "dist_to_road_m"]

train_selectors = band_list + frontier_band_list + ["label", "tYear"]
unbiased_selectors = band_list + frontier_band_list + ["label", "tYear", "unbiased"]

selectors = band_list + ["label", "tYear", "unbiased"]  



AEF bands: ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07'] ... total: 64


In [35]:
# ---- 1) Training samples (balanced stratified, merged across years) ----
train_fc = ee.FeatureCollection([
    stratified_samples_for_year(y, ROI, N_NEG_PER_YEAR, N_POS_PER_YEAR, SCALE, SEED)
    for y in TRAIN_YEARS
]).flatten()

train_desc = f"aef_train_balanced_{min(TRAIN_YEARS)}_{max(TRAIN_YEARS)}_v5" + ("_stablelabel" if USE_STABLE_LABEL else "")
train_task = export_fc_to_drive(
    train_fc,
    description=train_desc,
    filename_prefix=train_desc,
    selectors=train_selectors
)
print("Started TRAIN export:", train_desc)

# ---- 2) Unbiased eval samples (forest-only, random sample) ----
unb_fc = unbiased_forest_samples(UNBIASED_EVAL_YEAR, ROI, N_UNBIASED_FOREST_PIXELS, SCALE, SEED)

unb_desc = f"aef_unbiased_forest_eval_{UNBIASED_EVAL_YEAR}_v5" + ("_stablelabel" if USE_STABLE_LABEL else "")
unb_task = export_fc_to_drive(
    unb_fc,
    description=unb_desc,
    filename_prefix=unb_desc,
    selectors=unbiased_selectors
)
print("Started UNBIASED export:", unb_desc)

print("\nNow go to the Earth Engine Code Editor -> Tasks tab to monitor, or check Google Drive after completion.")

Started TRAIN export: aef_train_balanced_2018_2020_v5
Started UNBIASED export: aef_unbiased_forest_eval_2022_v5

Now go to the Earth Engine Code Editor -> Tasks tab to monitor, or check Google Drive after completion.


In [39]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.frozen import FrozenEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    roc_auc_score, average_precision_score, log_loss, brier_score_loss,
    confusion_matrix, accuracy_score, balanced_accuracy_score
)


# ===================== PATHS =====================
TRAIN_CSV = "/app/data/aef_train_balanced_2018_2020_v5.csv"   
UNBIASED_CSV =  "/app/data/aef_unbiased_forest_eval_2022_v5.csv"   
# ===================== SETTINGS =====================
TRAIN_YEARS = [2018, 2019, 2020]   # time split
TEST_YEAR   = 2020

# Alert budgets (map-style evaluation)
TOP_K_LIST = [1, 2, 5, 10]  # %

# ===================== LOAD =====================
df = pd.read_csv(TRAIN_CSV)
df_u = pd.read_csv(UNBIASED_CSV)

feature_cols = [f"A{i:02d}" for i in range(64)] + ["dist_to_nonforest_m", "dist_to_road_m"]

assert all(c in df.columns for c in feature_cols), "Train CSV missing A00..A63"
assert all(c in df_u.columns for c in feature_cols), "Unbiased CSV missing A00..A63"

# ===================== METRICS =====================
def eval_probs(y, p, name=""):
    p = np.clip(p, 1e-6, 1 - 1e-6)
    print(f"\n=== {name} ===")
    print("n:", len(y), "pos rate:", float(y.mean()))
    print("ROC-AUC:", roc_auc_score(y, p))
    print("PR-AUC:", average_precision_score(y, p))
    print("LogLoss:", log_loss(y, p))
    print("Brier:", brier_score_loss(y, p))

def summarize_at_threshold(y, p, thr, name=""):
    p = np.clip(p, 1e-6, 1 - 1e-6)
    pred = (p >= thr).astype(int)
    cm = confusion_matrix(y, pred)
    acc = accuracy_score(y, pred)
    bacc = balanced_accuracy_score(y, pred)

    tn, fp = cm[0,0], cm[0,1]
    fn, tp = cm[1,0], cm[1,1]
    precision = tp / max(tp + fp, 1)
    recall    = tp / max(tp + fn, 1)

    print(f"\n--- {name} @ thr={thr:.4f} ---")
    print("Confusion:\n", cm)
    print("Accuracy:", acc)
    print("Balanced accuracy:", bacc)
    print("Precision:", precision)
    print("Recall:", recall)

def topk_report(y, p, name=""):
    p = np.asarray(p)
    y = np.asarray(y)
    total_pos = max(int(y.sum()), 1)
    for k in TOP_K_LIST:
        thr = float(np.quantile(p, 1 - k/100.0))
        sel = p >= thr
        capture = int(y[sel].sum()) / total_pos
        prec = float(y[sel].mean()) if sel.sum() else 0.0
        print(f"Top {k:>2}%: thr={thr:.4f}  capture={capture:.3f}  precision={prec:.3f}")

# ===================== SPLITS =====================
train_df = df[df["tYear"].isin(TRAIN_YEARS)].copy()
test_df  = df[df["tYear"].eq(TEST_YEAR)].copy()

Xtr = train_df[feature_cols].to_numpy(np.float32)
ytr = train_df["label"].to_numpy(np.int32)

Xte = test_df[feature_cols].to_numpy(np.float32)
yte = test_df["label"].to_numpy(np.int32)

Xu  = df_u[feature_cols].to_numpy(np.float32)
yu  = df_u["label"].to_numpy(np.int32)

print("Train split:", TRAIN_YEARS, "n=", len(ytr), "pos rate=", ytr.mean())
print("Test year:", TEST_YEAR, "n=", len(yte), "pos rate=", yte.mean())
print("Unbiased year:", int(df_u["tYear"].iloc[0]) if "tYear" in df_u.columns else "?", "n=", len(yu), "pos rate=", yu.mean())

# ===================== 1) LOGISTIC REGRESSION =====================
logit = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        solver="lbfgs",
        max_iter=5000,
        C=1.0
    ))
])

logit.fit(Xtr, ytr)

# Time-split eval (balanced-ish)
p_te = logit.predict_proba(Xte)[:, 1]
eval_probs(yte, p_te, name="Logit: time-split test (balanced)")
summarize_at_threshold(yte, p_te, thr=0.5, name="Logit time-split (thr=0.5)")
topk_report(yte, p_te, name="Logit time-split")

# Unbiased eval (natural base rate)
p_u = logit.predict_proba(Xu)[:, 1]
eval_probs(yu, p_u, name="Logit: unbiased forest-only")
summarize_at_threshold(yu, p_u, thr=0.5, name="Logit unbiased (thr=0.5)")
topk_report(yu, p_u, name="Logit unbiased")

# ---- Calibrate on unbiased set (prefit model) ----
logit_frozen = FrozenEstimator(logit)  # logit is already fit on (Xtr, ytr)
logit_cal = CalibratedClassifierCV(logit_frozen, method="sigmoid", cv=5)
logit_cal.fit(Xu, yu)

p_u_cal = logit_cal.predict_proba(Xu)[:, 1]
eval_probs(yu, p_u_cal, name="Logit: unbiased (calibrated)")
summarize_at_threshold(yu, p_u_cal, thr=0.5, name="Logit calibrated unbiased (thr=0.5)")
topk_report(yu, p_u_cal, name="Logit calibrated unbiased")

# ===================== Export w,b for menagerie =====================
# IMPORTANT: if you use StandardScaler, the effective w,b in ORIGINAL A-space
# are different from clf.coef_. We compute the equivalent w,b in raw feature space.
scaler = logit.named_steps["scaler"]
clf = logit.named_steps["clf"]

w_scaled = clf.coef_.ravel()
b_scaled = float(clf.intercept_[0])

# Convert to raw-space weights: w_raw = w_scaled / scale_
# intercept_raw = b_scaled - sum(w_scaled * mean_/scale_)
w_raw = w_scaled / scaler.scale_
b_raw = b_scaled - float(np.sum(w_scaled * (scaler.mean_ / scaler.scale_)))

w_raw_str = ",".join([f"{v:.10g}" for v in w_raw])
print("\n=== MENAGERIE PARAMS (LOGIT, RAW SPACE) ===")
print("b:", f"{b_raw:.10g}")
print("w:", w_raw_str[:160], "... (len=", len(w_raw_str), ")")

# ===================== 2) LINEAR SVM BASELINE (CALIBRATED) =====================
# LinearSVC gives margins; calibrate to probabilities for comparison/map.
svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(C=1.0, max_iter=8000))
])
svm.fit(Xtr, ytr)

svm_frozen = FrozenEstimator(svm)  # svm is already fit on (Xtr, ytr)
svm_cal = CalibratedClassifierCV(svm_frozen, method="sigmoid", cv=5)
svm_cal.fit(Xu, yu)

p_u_svm = svm_cal.predict_proba(Xu)[:, 1]
eval_probs(yu, p_u_svm, name="SVM (calibrated): unbiased forest-only")
summarize_at_threshold(yu, p_u_svm, thr=0.5, name="SVM calibrated unbiased (thr=0.5)")
topk_report(yu, p_u_svm, name="SVM calibrated unbiased")

print("\nDone.")


Train split: [2018, 2019, 2020] n= 23096 pos rate= 0.35053688950467615
Test year: 2020 n= 8057 pos rate= 0.37942162095072607
Unbiased year: 2022 n= 14026 pos rate= 0.029516612006274062

=== Logit: time-split test (balanced) ===
n: 8057 pos rate: 0.37942162095072607
ROC-AUC: 0.9576175335296042
PR-AUC: 0.918954808358895
LogLoss: 0.25535842366918876
Brier: 0.0771487339442452

--- Logit time-split (thr=0.5) @ thr=0.5000 ---
Confusion:
 [[4560  440]
 [ 414 2643]]
Accuracy: 0.894005212858384
Balanced accuracy: 0.8882865554465162
Precision: 0.8572818683100876
Recall: 0.8645731108930323
Top  1%: thr=0.9977  capture=0.026  precision=1.000
Top  2%: thr=0.9957  capture=0.052  precision=0.975
Top  5%: thr=0.9890  capture=0.128  precision=0.968
Top 10%: thr=0.9732  capture=0.254  precision=0.965

=== Logit: unbiased forest-only ===
n: 14026 pos rate: 0.029516612006274062
ROC-AUC: 0.9449728039056188
PR-AUC: 0.37074392976545834
LogLoss: 0.268887388241014
Brier: 0.08268332289189215

--- Logit unbiased

In [40]:
# After fitting logit
names = feature_cols
w = logit.named_steps["clf"].coef_.ravel()

# Convert to comparable "importance": abs(weight) in standardized space
imp = pd.Series(np.abs(w), index=names).sort_values(ascending=False)
print(imp.head(20))


dist_to_nonforest_m    2.936693
A54                    1.890045
A61                    1.305763
A43                    1.203202
A24                    1.135789
A53                    1.006012
A33                    0.994663
A21                    0.990465
A35                    0.986301
A50                    0.974518
A30                    0.936797
A09                    0.836124
A62                    0.827375
A52                    0.791289
A25                    0.715737
A01                    0.702006
A56                    0.645595
A15                    0.583833
A07                    0.577139
A63                    0.577114
dtype: float64


In [41]:
def fit_eval(cols, label):
    Xtr_ = train_df[cols].to_numpy(np.float32)
    Xte_ = test_df[cols].to_numpy(np.float32)

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(solver="lbfgs", max_iter=5000, C=1.0))
    ])
    model.fit(Xtr_, ytr)

    # unbiased
    Xu_ = df_u[cols].to_numpy(np.float32)
    pu_ = model.predict_proba(Xu_)[:, 1]
    print("\n===", label, "===")
    print("Unbiased ROC:", roc_auc_score(yu, pu_))
    print("Unbiased PR :", average_precision_score(yu, pu_))
    topk_report(yu, pu_, name=label)

embed_cols = [f"A{i:02d}" for i in range(64)]
ctx_cols = ["dist_to_nonforest_m", "dist_to_road_m"]

fit_eval(embed_cols, "Embeddings only")
fit_eval(ctx_cols,   "Context only")
fit_eval(embed_cols + ctx_cols, "Embeddings + Context")



=== Embeddings only ===
Unbiased ROC: 0.9457982158396754
Unbiased PR : 0.35327331651499805
Top  1%: thr=0.9657  capture=0.174  precision=0.511
Top  2%: thr=0.9330  capture=0.297  precision=0.438
Top  5%: thr=0.7993  capture=0.529  precision=0.312
Top 10%: thr=0.5136  capture=0.756  precision=0.223

=== Context only ===
Unbiased ROC: 0.8642037219219756
Unbiased PR : 0.11724599106332922
Top  1%: thr=0.6503  capture=0.056  precision=0.163
Top  2%: thr=0.6500  capture=0.101  precision=0.149
Top  5%: thr=0.6496  capture=0.222  precision=0.131
Top 10%: thr=0.6483  capture=0.362  precision=0.107

=== Embeddings + Context ===
Unbiased ROC: 0.9449720053774658
Unbiased PR : 0.37074507637140347
Top  1%: thr=0.9788  capture=0.179  precision=0.525
Top  2%: thr=0.9568  capture=0.307  precision=0.452
Top  5%: thr=0.8767  capture=0.543  precision=0.321
Top 10%: thr=0.6605  capture=0.758  precision=0.224


In [42]:
for col in ["dist_to_nonforest_m", "dist_to_road_m"]:
    s0 = df_u[df_u["label"]==0][col].dropna()
    s1 = df_u[df_u["label"]==1][col].dropna()
    print("\n", col)
    print("mean label0:", float(s0.mean()), "median0:", float(s0.median()))
    print("mean label1:", float(s1.mean()), "median1:", float(s1.median()))



 dist_to_nonforest_m
mean label0: 5073.617819309574 median0: 2701.5541623443787
mean label1: 702.0927946264575 median1: 463.31271652791656

 dist_to_road_m
mean label0: 18122.892275896695 median0: 14800.239870454232
mean label1: 15483.844292893737 median1: 8231.379779855364


In [27]:
#####spatial holdout

In [28]:

def add_lonlat_and_tiles(df: pd.DataFrame, tile_km: float = 50.0) -> pd.DataFrame:
    """
    Adds columns: lon, lat, tile_x, tile_y, tile_id
    tile_km: approx tile size in km for spatial blocking (try 25, 50, 100).
    """
    def parse_pt(g):
        if pd.isna(g):
            return np.nan, np.nan
        obj = json.loads(g)
        # Expect {"type":"Point","coordinates":[lon,lat]}
        lon, lat = obj["coordinates"]
        return lon, lat

    coords = df[".geo"].apply(parse_pt)
    df = df.copy()
    df["lon"] = coords.apply(lambda t: t[0])
    df["lat"] = coords.apply(lambda t: t[1])

    # Approx degrees per km
    deg_per_km_lat = 1.0 / 111.32
    # lon degrees shrink by cos(lat)
    deg_per_km_lon = deg_per_km_lat / np.cos(np.deg2rad(df["lat"].clip(-89.9, 89.9)))

    tile_h = tile_km * deg_per_km_lat
    tile_w = tile_km * deg_per_km_lon

    # Use global origin (0,0) grid
    df["tile_x"] = np.floor(df["lon"] / tile_w).astype("Int64")
    df["tile_y"] = np.floor(df["lat"] / tile_h).astype("Int64")
    df["tile_id"] = df["tile_x"].astype(str) + "_" + df["tile_y"].astype(str)

    return df


In [43]:


def eval_probs(y, p, name=""):
    p = np.clip(p, 1e-6, 1 - 1e-6)
    print(f"\n=== {name} ===")
    print("n:", len(y), "pos rate:", float(y.mean()))
    print("ROC-AUC:", roc_auc_score(y, p))
    print("PR-AUC:", average_precision_score(y, p))
    print("LogLoss:", log_loss(y, p))
    print("Brier:", brier_score_loss(y, p))

def spatial_holdout_eval(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    feature_cols: list[str],
    tile_km: float = 50.0,
    test_tile_frac: float = 0.25,
    seed: int = 42,
    name: str = ""
):
    # Add tiles
    tr = add_lonlat_and_tiles(train_df, tile_km=tile_km)
    te = add_lonlat_and_tiles(test_df, tile_km=tile_km)

    # Hold out a subset of tiles from *training* years (spatial holdout)
    rng = np.random.default_rng(seed)
    tiles = tr["tile_id"].dropna().unique()
    rng.shuffle(tiles)
    n_test_tiles = max(1, int(len(tiles) * test_tile_frac))
    holdout_tiles = set(tiles[:n_test_tiles])

    tr_in  = tr[~tr["tile_id"].isin(holdout_tiles)]
    tr_out = tr[ tr["tile_id"].isin(holdout_tiles)]

    print("\n--- Spatial split setup ---")
    print("tile_km:", tile_km, "unique tiles:", len(tiles), "holdout tiles:", len(holdout_tiles))
    print("train-in n:", len(tr_in), "pos:", tr_in["label"].mean())
    print("train-out n:", len(tr_out), "pos:", tr_out["label"].mean())

    # Fit on in-tiles
    X_in = tr_in[feature_cols].to_numpy(np.float32)
    y_in = tr_in["label"].to_numpy(np.int32)

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(solver="lbfgs", max_iter=5000, C=1.0))
    ])
    model.fit(X_in, y_in)

    # Evaluate on held-out tiles (same years distribution, new space)
    X_out = tr_out[feature_cols].to_numpy(np.float32)
    y_out = tr_out["label"].to_numpy(np.int32)
    p_out = model.predict_proba(X_out)[:, 1]
    eval_probs(y_out, p_out, name=f"{name} TRAIN-years spatial holdout tiles")

    # Evaluate on unbiased forest-only (usually 2022) as well
    X_te = te[feature_cols].to_numpy(np.float32)
    y_te = te["label"].to_numpy(np.int32)
    p_te = model.predict_proba(X_te)[:, 1]
    eval_probs(y_te, p_te, name=f"{name} UNBIASED (forest-only)")

    return model, holdout_tiles


In [44]:


# Choose which features:
embed_cols = [f"A{i:02d}" for i in range(64)]
ctx_cols = ["dist_to_nonforest_m", "dist_to_road_m"]
feature_cols = embed_cols + ctx_cols  # or embed_cols only

model_spatial, tiles = spatial_holdout_eval(
    train_df=train_df,
    test_df=df_u,                 # unbiased forest-only
    feature_cols=feature_cols,
    tile_km=50.0,                 # try 25, 50, 100
    test_tile_frac=0.25,
    seed=42,
    name="LOGIT (emb+ctx)"
)


/tmp/ipykernel_43555/1330842671.py:26: UserWarning: you are shuffling a 'StringArray' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  rng.shuffle(tiles)



--- Spatial split setup ---
tile_km: 50.0 unique tiles: 33 holdout tiles: 8
train-in n: 20890 pos: 0.36065102920057446
train-out n: 2206 pos: 0.2547597461468722

=== LOGIT (emb+ctx) TRAIN-years spatial holdout tiles ===
n: 2206 pos rate: 0.2547597461468722
ROC-AUC: 0.9734254184308734
PR-AUC: 0.9132593618513314
LogLoss: 0.1797126790459221
Brier: 0.056742638313342274

=== LOGIT (emb+ctx) UNBIASED (forest-only) ===
n: 14026 pos rate: 0.029516612006274062
ROC-AUC: 0.9456202327869271
PR-AUC: 0.37546526177553585
LogLoss: 0.2640176756058159
Brier: 0.08129834279635728
